# What is stixdcpy
stixdcpy is a python package that facilitates access and analysis of STIX data. It provides APIs to query and download data from STIX data center (https://datacenter.stix.i4ds.net/) and s a set of tools for visualizing data and performing common analysis tasks.  With stixdcpy, users can query and download the following types of data from STIX data center:

- Quick-look light curves
- Housekeeping data
- Science data
- Energy calibration data
- Auxilary data
- STIX solar flare list



# Installation
```sh 
pip install stixdcpy
```
stixdcpy can be also installed from the stixdcpy github repository:
```sh 
pip install git+https://github.com/i4ds/stixdcpy.git
```

# Tutorial and documentation

- [Tutorial on Google Colab](https://colab.research.google.com/drive/17fQfbWjL0s0TpblbPL1Ysy_zFXj40FBf?usp=sharing)
- [Documentation](https://drhlxiao.github.io/stixdcpy/)

# Reporting Issues and Contributing

Open an issue on the GitHub repo to report a problem. Pull requests welcome.
Github repo: https://github.com/i4ds/stixdcpy



# Install dependencies 

In [ ]:
#Install stixdcpy from pypi
#!pip install stixdcpy
# Or install from stixdcpy github repository

!pip install git+https://github.com/i4ds/stixdcpy.git
!pip install numpy matplotlib pandas astropy joblib simplejson

In [ ]:
import numpy as np
import pandas as pd

from pprint import pprint
from datetime import timedelta
from matplotlib import pyplot as plt
from astropy.table import QTable
from stixdcpy.quicklook import LightCurves
from stixdcpy.energylut import EnergyLUT
from stixdcpy import auxiliary as aux
from stixdcpy.net import FitsQuery as fq
from stixdcpy.net import JSONRequest as jreq
from stixdcpy import instrument as inst
from stixdcpy.science import ScienceL1, Spectrogram, spec_fits_crop, spec_fits_concatenate, fits_time_to_datetime
from stixdcpy.housekeeping import Housekeeping
from astropy.io import fits
from stixdcpy import detector_view as dv
from stixdcpy import spectrogram  as cspec
#from google.colab import data_table
#data_table.enable_dataframe_formatter()
%matplotlib notebook

# Request Quick-look  data from STIX data center 

Let's download some light curves from STIX data center

In [ ]:
lc = LightCurves.from_sdc(start_utc='2022-08-30T05:00:00', end_utc='2022-08-30T10:00:00', ltc=True)


where: 
* start_utc -  start time (onboard time); 
* end_utc - end time ; 
* ltc - an optional argument.  Perform light time correction to light curves if it is True

It returns a data object



Preview the QL lightcurves

In [ ]:
lc.peek()

Access the data:

In [ ]:
lc.data.keys()


In [ ]:
#pprint(lc.data['counts'])

In [ ]:
fig=plt.figure()
plt.plot(lc.time, lc.counts[0,:])

# Housekeep data

Fetching housekeeping data from STIX data center:



In [ ]:
hk=Housekeeping.from_sdc(start_utc='2021-09-06T12:00:00', end_utc='2021-09-06T14:00:00')
pprint(hk.param_names)


Plot housekeeping data time series:

In [ ]:
hk.plot('NIX00078,NIX00079,NIX00080,NIX00081')


You need to provide the names of parameters to be plotted.

One could also plot time series of parameters using plt

In [ ]:
plt.step( hk.data['datetime'], hk.data['raw_values']['NIXD0023'])
plt.xlabel('UTC')
plt.ylabel('Mode')

# Calibration data

Request calibration data from STIX data center

In [ ]:
elut=EnergyLUT.request('2021-09-03T00:00:00')

Print information on the onboard elut and the calibration run peformed closest to the specified time:

In [ ]:
elut.info()


Print energy calibration factors 

In [ ]:
elut.data

Print the real energy range of bins of a pixel

In [ ]:
lut=elut.get_pixel_true_ebins(pixel=34)

pd.DataFrame(lut)

# S/C Ephemeris 

Fetch s/c ephemeris data from STIX data center: 

In [ ]:
emph=aux.Ephemeris.from_sdc(start_utc='2022-01-01T00:00:00', end_utc='2022-03-31T01:00:00', steps=100)

Preview the data

In [ ]:
emph.peek()

In [ ]:
pprint(emph.data['orbit'])

# STIX pointing

In [ ]:
fov=aux.Pointing.from_sdc('2022-03-19T00:00:00')
fov.peek()


The web tool at the link https://datacenter.stix.i4ds.net/view/ancillary provides more detailed information of STIX pointing

In [ ]:
#Let's plot the data
pprint(fov.data)

# Solar flare light time difference and viewing angle calculator

This tool allows calculation of the light time difference between a flare and observers at earth or solar orbiter, as well as the viewing angles. 

In [ ]:
flare_aux=jreq.request_flare_light_time_and_angle(utc='2022-03-01T00:00:00', flare_x=100, flare_y=1000, observer='stix')


pprint(flare_aux)

where:        

* utc -    observation time
* flare_x -  flare helio-projective longitude in units of arcsec as seen by the observer 
* flare_y -  flare helio-projective latitude in units of arcsec 
* observer: It can be either "earth" or "stix" . The default value is "earth". 

A data object is returnned


The tool is available at the link
https://datacenter.stix.i4ds.net/view/ancillary


# Access and preview science data

## level-1 science data


Let's download a level-1 science data file from STIX data center

In [ ]:
l1_obj=ScienceL1.from_sdc(request_id=2208270251)

Here **2112240046** is the requests ID (also called unique id), which can  can be found on the science data web page https://datacenter.stix.i4ds.net/view/list/bsd.   
**ltc** is an optional argument. Light time correction will be performed if ltc = True. 
**l1_obj** is a data object 

Access the FITS Header Data Unit List attached to the data object:

In [ ]:
hdul=l1_obj.hdul
hdul.info()

Note that FITS hdul is only one of the data attributes.  

Let's print the hdul header

In [ ]:
hdul['PRIMARY'].header

Preview science data 

In [ ]:
l1_obj.peek('spg,lc, spec,tbin, qllc')


Peform dead time correction to the data

In [ ]:
corrected_data=l1_obj.correct_dead_time()

Print the array dimensions

In [ ]:
corrected_data.keys()
for key, value in corrected_data.items():
    print(key, value.shape)

Plot the live ratios of detectors: 

In [ ]:
plt.figure()
plt.plot(l1_obj.time, corrected_data['live_ratio'])
plt.xlabel('Time (s)')
plt.ylabel('Live ratio of detector groups')

Plot pixel and energy integerated corrected count rate:

In [ ]:
plt.figure()
plt.plot(l1_obj.time, np.sum(corrected_data['corrected_rates'], axis=(2,3) ))


Visualize counts with the module DetectorView

In [ ]:
pixel_counts = l1_obj.pixel_total_counts
det_view = dv.DetectorView(pixel_counts)
det_view.display()

In [ ]:
l1_obj.ebins_high

In [ ]:
#print(l1_obj.url)
#link to science browser 

## Download and plot spectrograms

Let's fetch a spectrogram data file from STIX data center

In [ ]:
request_id=	2210223333
l4d=Spectrogram.from_sdc(request_id=request_id)


In [ ]:
l4d.hdul.info()
l4d.peek()

## Crop or combine spectrograms

To select a shorter time interval from a long-duration spectrogram, or to combine two spectrograms into a single FITS file

In [ ]:
plt.plot(l4d.datetime, np.sum(l4d.counts[:,4:10],axis=1))
plt.title(l4d.T0_utc)

In [ ]:
tstart = '2022-10-22T23:06:00'
tend = '2022-10-22T23:15:00'

In [ ]:
cropped_file = spec_fits_crop(l4d.filename, tstart, tend)

A new file containing the cropped spectrogram has been created. It can be accessed at its location:

In [ ]:
cropped_file

In [ ]:
cropped_spec = Spectrogram(cropped_file, None)
plt.plot(cropped_spec.datetime, np.sum(cropped_spec.counts[:,4:10],axis=1))
plt.title(cropped_spec.T0_utc)

Similarly, two spectrogram files can be combined, and simultaneously cropped to a particular time range if desired.

In [ ]:
spec_res=fq.query('2022-04-19T20:00:00', '2022-04-19T23:00:00', product_type='xray-spec')
file_locations = spec_res.fetch()

In [ ]:
tstart='2022-04-19T20:05:00'
combined_file = spec_fits_concatenate(file_locations[0], file_locations[1], tstart=tstart)

A new file containing the combined spectrogram has been created. It can be accessed at its location:

In [ ]:
combined_file

In [ ]:
with fits.open(combined_file) as f:
    primary, _, data, _ = f
    spectimes = fits_time_to_datetime(primary.header, data.data)
    plotdata = np.sum(data.data.counts[:,4:10],axis=1)

In [ ]:
import matplotlib.dates as mdates
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
plt.gca().xaxis.set_major_locator(mdates.HourLocator(interval=1))
plt.plot(spectimes.datetime,plotdata)
plt.gcf().autofmt_xdate()
plt.title(spectimes[0].isot)

In [ ]:
spec = cspec.Spectrogram.from_sdc('2022-03-23T04:14:00', '2022-03-23T04:41:00')

In [ ]:
spec.peek()

# Search and download FITS files from STIX data center

Set download location

In [ ]:
fq.chdir('./downloads')



In [ ]:
res=fq.query('2022-02-10T05:00:00', '2022-02-18T07:00:00', product_type='xray-cpd',filter='BKG', level='L1A')


Arguments: 
* **product_type**   options:  


| option        	| Data type             	|
|--------------	|----------------------	|
| any      	| Any type (default value)          	|
| science      	| Any science          	|
| quicklook    	| Any quick-look       	|
| housekeeping 	| Housekeeping         	|
| xray-rpd     	| Raw pixel data       	|
| xray-cpd     	| Pixel data           	|
| xray-scpd    	| Summed pixel data    	|
| xray-vis     	| Visibility           	|
| xray-spec    	| Spectrogram          	|
| ql-lc        	| QL light curve       	|
| ql-var       	| QL variance          	|
| ql-bkg       	| QL background        	|
| ql-spec      	| QL spectra           	|
| ql-ffl       	| QL flare location    	|
| ql-cal       	| Calibration spectrum 	|
| aspect       	| Bulk aspect          	|
| auxiliary    	| Auxiliary data       	|


* filter: optional. 
    - Use to filter FITS files. Only science data with descriptions containing the specified string will be selected. 
    set it to None if you  don't want to filter the results



An instance of FitQueryResult  is returned. 



Convert the query result to pandas dataframe: 


In [ ]:
df=res.to_pandas()
df


One could download all the FITS file from stix data center by using the function "fetch": 

In [ ]:
fits_list=res.fetch()

Note that if a FITS file is found on the local disk, it will not be downloaded again from STIX data center. 

Open all the download FITS file: 

In [ ]:
hduls=res.open_fits()


hduls is a list of hdu objects

Print file info

In [ ]:
res.fits_info()

# Download flare list from STIX data center

In [ ]:
flares=jreq.fetch_flare_list('2021-04-08T00:00:00', '2021-04-12T00:00:00', sort='goes')
pprint(flares)


Parameters:

* start_UTC: str
  - start time
* end_UTC: str
  - end time

* sorting key: str, options:
 
  - LC0, flare peak counts of the 4 - 10 keV light curve 

  - LC1, flare peak counts of 10 - 10 keV light curves
  - LC2, similar to above, but of the third energy band
  - LC3,  similar to above, but of the fourth energy band
  - LC4,  similar to above, but of the fifth  energy band
  - time,  flare observation time
  - goes, goes class 

The number of flares can be downloaded each time is limited to 5000.

Display the list in  table view:


In [ ]:
df=pd.DataFrame(flares)
df

# Find science data requests

In [ ]:
sci_list=jreq.query_science('2021-10-01T00:00:00', '2021-10-02T00:00:00', request_type='all')

request_type, optional, if it is given, it will only return the specify the science request type
request_type options: 
* 'xray-cpd': pixel data
* 'xray-scpd': compression level-2 pixel data
* 'xray-spec': spectrograms
* 'xray-vis': visibilities 
* 'aspect': aspect request
* 'all':  any types


In [ ]:
df=pd.DataFrame(sci_list)
df